In [1]:
import os
from http.cookiejar import MozillaCookieJar

import cloudscraper
from bs4 import BeautifulSoup

from importlib import reload

import src.url_manager as url
import src.shoot as sh

reload(url)
reload(sh)

cookie_file = os.path.expanduser("~/cookies.txt")
base_url = 'www.kink.com'

In [2]:
file = open(cookie_file, "r")
file.read()

'# Netscape HTTP Cookie File\n# https://curl.haxx.se/rfc/cookie_spec.html\n# This is a generated file! Do not edit.\n\naccounts.google.com\tFALSE\t/\tTRUE\t1702487488\tOTZ\t7293191_52_52_123900_48_436380\n.google.com\tTRUE\t/\tFALSE\t1762969345\tSID\tdAgea0-yQmkY4Ez8jsT2eIUrmtZkFx8KxtvRfOQ_WIvxf81E9i_NMG_2cG8u9v0lFuDOaA.\n#HttpOnly_.google.com\tTRUE\t/\tTRUE\t1762969345\t__Secure-1PSID\tdAgea0-yQmkY4Ez8jsT2eIUrmtZkFx8KxtvRfOQ_WIvxf81En4jGliMupZ9ibzdKMwSteA.\n#HttpOnly_.google.com\tTRUE\t/\tTRUE\t1762969345\t__Secure-3PSID\tdAgea0-yQmkY4Ez8jsT2eIUrmtZkFx8KxtvRfOQ_WIvxf81EnfcaxcKM4ds37iOpMp_Oyw.\n#HttpOnly_.google.com\tTRUE\t/\tFALSE\t1762969345\tHSID\tAhIRzp4gGmxYI5gWY\n#HttpOnly_.google.com\tTRUE\t/\tTRUE\t1762969345\tSSID\tA_yvhPadbrCXQRG-m\n.google.com\tTRUE\t/\tFALSE\t1762969345\tAPISID\t2QhCIfezeVlGsPhW/ANyv31OsBQIpkHkLV\n.google.com\tTRUE\t/\tTRUE\t1762969345\tSAPISID\toBntKEExJBPFpGDQ/A_wbaXzrwXLQmJjE7\n.google.com\tTRUE\t/\tTRUE\t1762969345\t__Secure-1PAPISID\toBntKEExJBPFpGDQ/A

In [3]:
site = url.UrlManager(url.UrlManager.HttpType.HTTPS, base_url, ['my', 'favorite-scenes', 'page', '2'])
site.get_url()

'https://www.kink.com/my/favorite-scenes/page/2/'

In [4]:
session = cloudscraper.CloudScraper()
cookie = MozillaCookieJar(cookie_file)
cookie.load(ignore_expires=False, ignore_discard=False)
cookie

<MozillaCookieJar[Cookie(version=0, name='OTZ', value='7293191_52_52_123900_48_436380', port=None, port_specified=False, domain='accounts.google.com', domain_specified=False, domain_initial_dot=False, path='/', path_specified=False, secure=True, expires=1702487488, discard=False, comment=None, comment_url=None, rest={}, rfc2109=False), Cookie(version=0, name='ACCOUNT_CHOOSER', value='AFx_qI6Fiqjqm7dOpF_XfTNU-gEXI_HOWpUvPzHRclYhMYda8eXvZWecMdCysFq6QJAMGYPu3oq_oCzWpMZghK8zDsLV8uwONFzvpO6gxBwPv18PtX7Nm7Xly6RRKtNIUv43MT8LYNNW', port=None, port_specified=False, domain='accounts.google.com', domain_specified=False, domain_initial_dot=False, path='/', path_specified=False, secure=True, expires=1762969345, discard=False, comment=None, comment_url=None, rest={'HTTPOnly': ''}, rfc2109=False), Cookie(version=0, name='__Host-GAPS', value='1:ffWH0Jys1fGqKv3Zf4pxNaXCV6v5ozCmG-qKJ7JphpUVSNNTIu2Z7w9DyVfZZgINygR3mQHDSxFA2HmnvBjoxMkLxPlH3g:j45yziwlmYebi8KR', port=None, port_specified=False, domain='acco

In [5]:
type(cookie)

http.cookiejar.MozillaCookieJar

In [6]:
req = session.get(site.get_url(), cookies=cookie)
soup = BeautifulSoup(req.text, "html.parser")

In [7]:
last_page_of_favorites = soup.select('li.page-item:last-child > a:nth-child(1)')[0].get_text()
int(last_page_of_favorites)

92

In [8]:
with open("output.html", "w") as file:
    file.write(str(soup.prettify()))

In [9]:
url_shoot = [url.UrlManager(url.UrlManager.HttpType.HTTPS, base_url, el.get('href')) for el in
             soup.find_all('a', 'd-block')]
first = url_shoot[0]

In [10]:
first

<UrlManager: https://www.kink.com/shoot/34226/>

In [11]:
req = session.get(first.get_url(), cookies=cookie)
soup = BeautifulSoup(req.text, "html.parser")

In [12]:
soup.find_all('h1')[0].get_text() == '404' 

False

In [13]:
soup.find('h1', 'shoot-title').get_text()

'Katherine Kane: Champion Anal Electroslut Live Show!\n\ue800\n'

In [14]:
videos = {val.get('quality'): val.get('download') for val in soup.find_all('a', download=True, quality=True)}
videos

{'270': 'https://cdn.kink.com/electrosluts/members/34226/video/h264/full/34226_Katharine_Aiden_lo.mp4?nva=1701363234&token=70cf1bced6dcc94e2a1ac854457deac8',
 '360': 'https://cdn.kink.com/electrosluts/members/34226/video/h264/full/34226_Katharine_Aiden_md.mp4?nva=1701363234&token=d210cecaef0d0d0dfcd274ede464d66f',
 '720': 'https://cdn.kink.com/electrosluts/members/34226/video/h264/full/34226_Katharine_Aiden.mp4?nva=1701363234&token=49c0618691d98387539f821ae4972176'}

In [15]:
poster = soup.find('video', poster=True).get('poster')
poster

'https://imgopt.kink.com/imagedb/34226/indexImage/34226_index_qfhd_full.png?w=900&s=3377fff46ada3e3024b0e0c15bba844a36ea7e4c755055fa64613474bfce1b32'

In [16]:
zip_image = soup.find('a', 'zip-links', download=True).get('download')
zip_image

'https://cdn.kink.com/electrosluts/members/34226/pictures/hires/1%20Pretty%20Girls/electrosluts_34226_hi_res_pictures_1%20Pretty%20Girls.zip?nva=1701363234&token=32e9884a5f0e01d1ddfb7f447a83b3f9'

In [17]:
actors = soup.select('.names > a')
actors

[<a href="/model/11489/Aiden-Starr">Aiden Starr, </a>,
 <a href="/model/35830/Katharine-Cane">Katharine Cane </a>]